<a href="https://colab.research.google.com/github/Daniaarav2005/CoinSage/blob/Abhijeet/Hackathon_Illinois.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

abhijeetbeedikar_doge_data_path = kagglehub.da\taset_download('abhijeetbeedikar/doge-data')

print('Data source import complete.')


In [ ]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
pooled_data = pd.read_csv("/kaggle/input/doge-data/pooled_data.csv")
pooled_data["net"] = pooled_data["net"].apply(lambda x: 1 if x > 0 else 0)
pooled_data = pooled_data.sample(frac=1, random_state=42).reset_index(drop=True)
pooled_data = pooled_data.loc[:,["tweet","net"]]
X = pooled_data["tweet"]
y = pooled_data["net"]

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Initialize the tokenizer
tokenizer = Tokenizer(num_words=10000)  # Set max words for vocabulary

# Fit the tokenizer on the text data
tokenizer.fit_on_texts(X)

# Convert text to sequences of integers
X_sequences = tokenizer.texts_to_sequences(X)

# Pad the sequences to ensure they are all the same length
X_padded = pad_sequences(X_sequences, padding='post', maxlen=100)  # Adjust maxlen based on your dataset

# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_padded, y, test_size=0.2, random_state=42)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization

# Build the LSTM model
model = Sequential()

# Embedding layer (embeddings can be pre-trained like GloVe or random)
model.add(BatchNormalization())
model.add(Embedding(input_dim=10000, output_dim=256, input_length=200))
model.add(BatchNormalization())

# LSTM layer
model.add(LSTM(256, dropout=0.25, recurrent_dropout=0.2))
model.add(BatchNormalization())


# Output layer (binary classification)
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'],)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(min_delta = 0.01,monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=1024, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose = 1)


Epoch 1/20
153/153 ━━━━━━━━━━━━━━━━━━━━ 28s 186ms/step - binary_accuracy: 0.5315 - loss: 0.6914 - val_binary_accuracy: 0.5294 - val_loss: 0.6914
Epoch 2/20
153/153 ━━━━━━━━━━━━━━━━━━━━ 28s 186ms/step - binary_accuracy: 0.5301 - loss: 0.6914 - val_binary_accuracy: 0.5293 - val_loss: 0.6914
Epoch 3/20
153/153 ━━━━━━━━━━━━━━━━━━━━ 29s 187ms/step - binary_accuracy: 0.5315 - loss: 0.6913 - val_binary_accuracy: 0.5295 - val_loss: 0.6914
Epoch 4/20
153/153 ━━━━━━━━━━━━━━━━━━━━ 28s 184ms/step - binary_accuracy: 0.5317 - loss: 0.6913 - val_binary_accuracy: 0.5293 - val_loss: 0.6914
Epoch 5/20
153/153 ━━━━━━━━━━━━━━━━━━━━ 28s 186ms/step - binary_accuracy: 0.5276 - loss: 0.6917 - val_binary_accuracy: 0.5293 - val_loss: 0.6915
Epoch 6/20
153/153 ━━━━━━━━━━━━━━━━━━━━ 28s 184ms/step - binary_accuracy: 0.5308 - loss: 0.6913 - val_binary_accuracy: 0.5293 - val_loss: 0.6914
Epoch 7/20
153/153 ━━━━━━━━━━━━━━━━━━━━ 28s 186ms/step - binary_accuracy: 0.5303 - loss: 0.6915 - val_binary_accuracy: 0.5293 - va

In [ ]:
def model_predict(input:list[str]):
    # Convert new text to sequence and pad it
    new_text = input
    new_text_seq = tokenizer.texts_to_sequences(new_text)
    new_text_padded = pad_sequences(new_text_seq, padding='post', maxlen=100)

    # Predict using the trained model
    prediction = model.predict(new_text_padded)
    return f"Predicted labels: {prediction[0]}"


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


'Predicted labels: [0.52672195]'

In [ ]:
model.save("/kaggle/working/model.h5")